In [3]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

In [4]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 13:50:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/16 13:50:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local[2] appName=Create-DataFrame>

In [5]:
sc = spark.sparkContext

# Wczytanie pliku do RDD
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

# Transformacja: filtrujemy linie zawierające słowo 'Tadeusz'
lines_with_tadeusz = pan_tadeusz_file.filter(lambda line: 'Tadeusz' in line)

# Akcja: liczymy liczbę takich linii
count_tadeusz_lines = lines_with_tadeusz.count()

print(f"Liczba linii zawierających słowo 'Tadeusz': {count_tadeusz_lines}")

[Stage 0:>                                                          (0 + 2) / 2]

Liczba linii zawierających słowo 'Tadeusz': 182


In [6]:
# Transformacja: mapujemy każdą linię na jej długość
lines_with_length = pan_tadeusz_file.map(lambda line: (len(line), line))

# Akcja: wybieramy 3 najdłuższe linie
top_3_lines = lines_with_length.top(3)

# Wyświetlamy 3 najdłuższe linie
print("Trzy najdłuższe linie:")
for length, line in top_3_lines:
    print(f"{length} characters: {line}")

Trzy najdłuższe linie:
536 characters: O niebezpieczeństwach wynikających z nieporządnego obozowania — Odsiecz niespodziana — Smutne położenie szlachty — Odwiedziny kwestarskie są wróżbą ratunku — Major Płut zbytnią zalotnością ściąga na siebie burzę — Wystrzał z krócicy, hasło boju — Czyny Kropiciela, czyny i niebezpieczeństwa Maćka — Konewka zasadzką ocala Soplicowo — Posiłki jezdne, atak na piechotę — Czyny Tadeusza — Pojedynek dowódców przerwany zdradą — Wojski stanowczym manewrem przechyla szalę boju — Czyny krwawe Gerwazego — Podkomorzy zwycięzca wspaniałomyślny.
452 characters: Plany myśliwskie Telimeny — Ogrodniczka wybiera się na wielki świat i słucha nauk opiekunki — Strzelcy wracają — Wielkie zadziwienie Tadeusza — Spotkanie się powtórne w Świątyni dumania i zgoda ułatwiona za pośrednictwem mrówek — U stołu wytacza się rzecz o łowach — Powieść Wojskiego o Rejtanie i księciu Denassów, przerwana — Zagajenie układów między stronami, także przerwane — Zjawisko z kluczem — Kłótni

In [12]:
import re
import json

# Wczytanie pliku do RDD
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

# Wczytanie listy stopwords
stopwords = sc.textFile("polish.stopwords.txt").collect()  # Zbieramy stopwords w postaci listy

# Zamieniamy listę stopwords na zbiór dla szybszego sprawdzania przynależności
stopwords_set = set(stopwords)

# Funkcja czyszcząca tekst: usuwa znaki przestankowe i stopwords
def clean_text(line):
    # Usuwamy znaki przestankowe
    line = re.sub(r'[^\w\s]', '', line.lower())
    # Dzielimy tekst na słowa
    words = line.split()
    # Filtrujemy słowa, usuwając stopwords
    return [word for word in words if word not in stopwords_set]

# Transformacja: czyszczenie linii i spłaszczanie wyników do listy słów
words_rdd = pan_tadeusz_file.flatMap(clean_text)

# Akcja: liczymy wystąpienia słów (wywołana po wszystkich transformacjach)
word_counts = words_rdd.countByValue()

# Zapisz wynik do pliku JSON
with open("pan_tadeusz_bag_of_words.json", "w") as f:
    json.dump(word_counts, f)

# Najczęściej występujące słowo
most_frequent_word = max(word_counts, key=word_counts.get)
print(f"Najczęściej występujące słowo: '{most_frequent_word}' z {word_counts[most_frequent_word]} wystąpieniami.")


Najczęściej występujące słowo: 'rzekł' z 155 wystąpieniami.
